In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data = pd.read_csv('../datasets/AmesHousing.tsv', delimiter="\t")

In [3]:
def transform_features(data):
    return df

In [4]:
def select_features(df):
    return df[["Gr Liv Area", "SalePrice"]]

In [5]:
def train_and_test(df):
    train = df.iloc[:1460]
    test = df.iloc[1460:]
    
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error
    
    features = train.select_dtypes(include=['int', 'float']).drop(columns="SalePrice").columns
    lr = LinearRegression()
    lr.fit(train[features], train['SalePrice'])

    test_predictions = lr.predict(test[features])

    test_mse = mean_squared_error(test_predictions, test['SalePrice'])
    
    test_rmse = np.sqrt(test_mse)

    return test_rmse

filtered_df = select_features(data)
rmse = train_and_test(filtered_df)
rmse

57088.25161263909

# Clean the data and select transformations

In [6]:
#1 drop columns with more than 25% missing values
missing_val = data.isnull().sum()
drop_cols = missing_val[(missing_val/len(data)) > 0.05]
data = data.drop(columns=drop_cols.index)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 71 columns):
Order              2930 non-null int64
PID                2930 non-null int64
MS SubClass        2930 non-null int64
MS Zoning          2930 non-null object
Lot Area           2930 non-null int64
Street             2930 non-null object
Lot Shape          2930 non-null object
Land Contour       2930 non-null object
Utilities          2930 non-null object
Lot Config         2930 non-null object
Land Slope         2930 non-null object
Neighborhood       2930 non-null object
Condition 1        2930 non-null object
Condition 2        2930 non-null object
Bldg Type          2930 non-null object
House Style        2930 non-null object
Overall Qual       2930 non-null int64
Overall Cond       2930 non-null int64
Year Built         2930 non-null int64
Year Remod/Add     2930 non-null int64
Roof Style         2930 non-null object
Roof Matl          2930 non-null object
Exterior 1st       29

In [7]:
#2 drop text cols with more than 1 missing val
text_cols = data.select_dtypes(include=["object"]).isnull().sum()
text_cols_drop = text_cols[text_cols > 0]
data = data.drop(columns=text_cols_drop.index)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 64 columns):
Order              2930 non-null int64
PID                2930 non-null int64
MS SubClass        2930 non-null int64
MS Zoning          2930 non-null object
Lot Area           2930 non-null int64
Street             2930 non-null object
Lot Shape          2930 non-null object
Land Contour       2930 non-null object
Utilities          2930 non-null object
Lot Config         2930 non-null object
Land Slope         2930 non-null object
Neighborhood       2930 non-null object
Condition 1        2930 non-null object
Condition 2        2930 non-null object
Bldg Type          2930 non-null object
House Style        2930 non-null object
Overall Qual       2930 non-null int64
Overall Cond       2930 non-null int64
Year Built         2930 non-null int64
Year Remod/Add     2930 non-null int64
Roof Style         2930 non-null object
Roof Matl          2930 non-null object
Exterior 1st       29

In [8]:
#3 fill in numerical columns with less than 5% missing values with the mode
numerical_cols = data.select_dtypes(include=['int', 'float'])
num_missing_val = data.isnull().sum()
mode_replace_cols = num_missing_val[((num_missing_val/len(data)) < 0.05) &((num_missing_val/len(data)) > 0)] 
for col in mode_replace_cols.index:
    data[col].fillna(int(data[col].mode()), inplace=True)
data.isnull().sum().value_counts()

0    64
dtype: int64

In [9]:
#4 new features to built: 
# a) how old has it been at sale YEARS_sold
# b) how long has last remod been ago years_remod

data["years_sold"] = data["Yr Sold"] - data["Year Built"]
data["years_remod"] = data["Yr Sold"] - data["Year Remod/Add"]

data = data.drop(data[data["years_sold"] < 0].index)
data = data.drop(data[data["years_remod"] < 0].index)

In [10]:
data["years_sold"].value_counts().sort_index()

0      114
1      218
2       90
3       60
4       76
5       66
6       66
7       51
8       44
9       50
10      40
11      47
12      33
13      38
14      27
15      31
16      22
17      25
18      14
19      16
20       9
21       9
22      12
23      12
24       8
25      11
26      17
27       9
28      19
29      30
      ... 
97      11
98      10
99      13
100      5
101      1
102      1
103      3
104      1
106     10
107      5
108      7
109      5
110      5
111      1
112      2
113      1
114      2
115      1
117      3
118      2
119      2
120      1
122      1
125      1
126      1
127      2
128      2
129      2
135      1
136      1
Name: years_sold, Length: 127, dtype: int64

In [11]:
data["years_remod"].value_counts().sort_index()

0     238
1     193
2     104
3      97
4     111
5      91
6      83
7      75
8      85
9      71
10     66
11     62
12     59
13     57
14     46
15     37
16     38
17     33
18     18
19     21
20     18
21     15
22     13
23     18
24     10
25     14
26     17
27     18
28     22
29     26
     ... 
31     50
32     37
33     29
34     30
35     22
36     37
37     30
38     27
39     47
40     30
41     26
42     33
43     27
44     34
45     22
46     25
47     28
48     26
49     27
50     23
51     31
52     29
53     30
54     21
55     23
56    112
57     79
58     76
59     78
60     42
Name: years_remod, Length: 61, dtype: int64

In [12]:
## Drop columns that aren't useful for ML
data = data.drop(["PID", "Order"], axis=1)

## Drop columns that leak info about the final sale
data = data.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

In [13]:
def transform_features(data):
    #1 drop columns with more than 25% missing values
    missing_val = data.isnull().sum()
    drop_cols = missing_val[(missing_val/len(data)) > 0.05]
    data = data.drop(columns=drop_cols.index)
    
    #2 drop text cols with more than 1 missing val
    text_cols = data.select_dtypes(include=["object"]).isnull().sum()
    text_cols_drop = text_cols[text_cols > 0]
    data = data.drop(columns=text_cols_drop.index)

    #3 fill in numerical columns with less than 5% missing values with the mode
    numerical_cols = data.select_dtypes(include=['int', 'float'])
    num_missing_val = data.isnull().sum()
    mode_replace_cols = num_missing_val[((num_missing_val/len(data)) < 0.05) &((num_missing_val/len(data)) > 0)] 
    for col in mode_replace_cols.index:
        data[col].fillna(int(data[col].mode()), inplace=True)
        
    #4 new features to built: 
    # a) how old has it been at sale YEARS_sold
    # b) how long has last remod been ago years_remod

    data["years_sold"] = data["Yr Sold"] - data["Year Built"]
    data["years_remod"] = data["Yr Sold"] - data["Year Remod/Add"]

    data = data.drop(data[data["years_sold"] < 0].index)
    data = data.drop(data[data["years_remod"] < 0].index)
    
    ## Drop columns that aren't useful for ML
    data = data.drop(["PID", "Order"], axis=1)

    ## Drop columns that leak info about the final sale
    data = data.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
    return data

data = pd.read_csv('../datasets/AmesHousing.tsv', delimiter="\t")
transform_df = transform_features(data)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

55275.367312413066

# Starting to select features

In [14]:
numerical_cols = data.select_dtypes(include=['int', 'float']).columns
cols = data[numerical_cols].corr()["SalePrice"].sort_values(ascending=False)[0:20]
cols

SalePrice         1.000000
Overall Qual      0.799262
Gr Liv Area       0.706780
Garage Cars       0.647877
Garage Area       0.640401
Total Bsmt SF     0.632280
1st Flr SF        0.621676
Year Built        0.558426
Full Bath         0.545604
Year Remod/Add    0.532974
Garage Yr Blt     0.526965
Mas Vnr Area      0.508285
TotRms AbvGrd     0.495474
Fireplaces        0.474558
BsmtFin SF 1      0.432914
Lot Frontage      0.357318
Wood Deck SF      0.327143
Open Porch SF     0.312951
Half Bath         0.285056
Bsmt Full Bath    0.276050
Name: SalePrice, dtype: float64

testing a model with numerical features and variable pearson correlation value, here: 0.4

In [15]:
def transform_features(data):
    #1 drop columns with more than 5% missing values
    missing_val = data.isnull().sum()
    drop_cols = missing_val[(missing_val/len(data)) > 0.05]
    data = data.drop(columns=drop_cols.index)
    
    #2 drop text cols with more than 1 missing val
    text_cols = data.select_dtypes(include=["object"]).isnull().sum()
    text_cols_drop = text_cols[text_cols > 0]
    data = data.drop(columns=text_cols_drop.index)

    #3 fill in numerical columns with less than 5% missing values with the mode
    numerical_cols = data.select_dtypes(include=['int', 'float'])
    num_missing_val = data.isnull().sum()
    mode_replace_cols = num_missing_val[((num_missing_val/len(data)) < 0.05) &((num_missing_val/len(data)) > 0)] 
    for col in mode_replace_cols.index:
        data[col].fillna(int(data[col].mode()), inplace=True)
        
    #4 new features to built: 
    # a) how old has it been at sale YEARS_sold
    # b) how long has last remod been ago years_remod

    data["years_sold"] = data["Yr Sold"] - data["Year Built"]
    data["years_remod"] = data["Yr Sold"] - data["Year Remod/Add"]

    data = data.drop(data[data["years_sold"] < 0].index)
    data = data.drop(data[data["years_remod"] < 0].index)
    
    ## Drop columns that aren't useful for ML
    data = data.drop(["PID", "Order"], axis=1)

    ## Drop columns that leak info about the final sale
    data = data.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
    return data

def select_features(df, corr_treshold):
    numerical_cols = df.select_dtypes(include=['int', 'float']).columns
    cols = df[numerical_cols].corr()["SalePrice"].sort_values(ascending=False)[df[numerical_cols].corr()["SalePrice"] > corr_treshold].index
    return df[cols]

def train_and_test(df):
    train = df.iloc[:1460]
    test = df.iloc[1460:]
    
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error
    
    features = train.select_dtypes(include=['int', 'float']).drop(columns="SalePrice").columns
    lr = LinearRegression()
    lr.fit(train[features], train['SalePrice'])

    test_predictions = lr.predict(test[features])

    test_mse = mean_squared_error(test_predictions, test['SalePrice'])
    
    test_rmse = np.sqrt(test_mse)

    return test_rmse

data = pd.read_csv('../datasets/AmesHousing.tsv', delimiter="\t")
transform_df = transform_features(data)
rmses = {}
filtered_df = select_features(transform_df, 0.4)
rmse = train_and_test(filtered_df)
rmse

36706.41086991501

identifying and adding categorical columns to the model

In [16]:
## Create a list of column names from documentation that are *meant* to be categorical
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

## Which categorical columns have we still carried with us? We'll test tehse 
transform_cat_cols = []
for col in nominal_features:
    if col in transform_df.columns:
        transform_cat_cols.append(col)
transform_cat_cols

['MS SubClass',
 'MS Zoning',
 'Street',
 'Land Contour',
 'Lot Config',
 'Neighborhood',
 'Condition 1',
 'Condition 2',
 'Bldg Type',
 'House Style',
 'Roof Style',
 'Roof Matl',
 'Exterior 1st',
 'Exterior 2nd',
 'Foundation',
 'Heating',
 'Central Air']

In [17]:
## How many unique values in each categorical column?
uniqueness_counts = transform_df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
## Aribtrary cutoff of 10 unique values (worth experimenting)
drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
transform_df = transform_df.drop(drop_nonuniq_cols, axis=1)

## Select just the remaining text columns and convert to categorical
text_cols = transform_df.select_dtypes(include=['object'])
for col in text_cols:
    transform_df[col] = transform_df[col].astype('category')
    
## Create dummy columns and add back to the dataframe!
transform_df = pd.concat([
    transform_df, 
    pd.get_dummies(transform_df.select_dtypes(include=['category']))
], axis=1)

Testing a model with numerical and categorical features (dummies) with 5 or less unique values

In [18]:
def transform_features(data):
    #1 drop columns with more than 5% missing values
    missing_val = data.isnull().sum()
    drop_cols = missing_val[(missing_val/len(data)) > 0.05]
    data = data.drop(columns=drop_cols.index)
    
    #2 drop text cols with more than 1 missing val
    text_cols = data.select_dtypes(include=["object"]).isnull().sum()
    text_cols_drop = text_cols[text_cols > 0]
    data = data.drop(columns=text_cols_drop.index)

    #3 fill in numerical columns with less than 5% missing values with the mode
    numerical_cols = data.select_dtypes(include=['int', 'float'])
    num_missing_val = data.isnull().sum()
    mode_replace_cols = num_missing_val[((num_missing_val/len(data)) < 0.05) &((num_missing_val/len(data)) > 0)] 
    for col in mode_replace_cols.index:
        data[col].fillna(int(data[col].mode()), inplace=True)
        
    #4 new features to built: 
    # a) how old has it been at sale YEARS_sold
    # b) how long has last remod been ago years_remod

    data["years_sold"] = data["Yr Sold"] - data["Year Built"]
    data["years_remod"] = data["Yr Sold"] - data["Year Remod/Add"]

    data = data.drop(data[data["years_sold"] < 0].index)
    data = data.drop(data[data["years_remod"] < 0].index)
    
    ## Drop columns that aren't useful for ML
    data = data.drop(["PID", "Order"], axis=1)

    ## Drop columns that leak info about the final sale
    data = data.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
    return data

def select_features(df, corr_treshold=0.4, unique_treshold=5):
    numerical_df = df.select_dtypes(include=['int', 'float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < corr_treshold].index, axis=1)
    ## Create a list of column names from documentation that are *meant* to be categorical
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

    ## Which categorical columns have we still carried with us? We'll test tehse 
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)
    
    ## How many unique values in each categorical column?
    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    ## Aribtrary cutoff of 10 unique values (worth experimenting)
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > unique_treshold].index
    df = df.drop(drop_nonuniq_cols, axis=1)

    ## Select just the remaining text columns and convert to categorical
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    
    ## Create dummy columns and add back to the dataframe!
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1)
    
    return df

def train_and_test(df):
    train = df.iloc[:1460]
    test = df.iloc[1460:]
    
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error
    
    features = train.select_dtypes(include=['int', 'float', 'uint8']).drop(columns="SalePrice").columns
    lr = LinearRegression()
    lr.fit(train[features], train['SalePrice'])

    test_predictions = lr.predict(test[features])

    test_mse = mean_squared_error(test_predictions, test['SalePrice'])
    
    test_rmse = np.sqrt(test_mse)

    return (test_rmse, features)

data = pd.read_csv('../datasets/AmesHousing.tsv', delimiter="\t")
transform_df = transform_features(data)
rmses = {}
filtered_df = select_features(transform_df, 0.4, 5)
rmse, features = train_and_test(filtered_df)

print(rmse)
print(features)

33029.341096815835
Index(['Overall Qual', 'Year Built', 'Year Remod/Add', 'Mas Vnr Area',
       'BsmtFin SF 1', 'Total Bsmt SF', '1st Flr SF', 'Gr Liv Area',
       'Full Bath', 'TotRms AbvGrd', 'Fireplaces', 'Garage Cars',
       'Garage Area', 'years_sold', 'years_remod', 'Street_Grvl',
       'Street_Pave', 'Lot Shape_IR1', 'Lot Shape_IR2', 'Lot Shape_IR3',
       'Lot Shape_Reg', 'Land Contour_Bnk', 'Land Contour_HLS',
       'Land Contour_Low', 'Land Contour_Lvl', 'Utilities_AllPub',
       'Utilities_NoSeWa', 'Utilities_NoSewr', 'Lot Config_Corner',
       'Lot Config_CulDSac', 'Lot Config_FR2', 'Lot Config_FR3',
       'Lot Config_Inside', 'Land Slope_Gtl', 'Land Slope_Mod',
       'Land Slope_Sev', 'Bldg Type_1Fam', 'Bldg Type_2fmCon',
       'Bldg Type_Duplex', 'Bldg Type_Twnhs', 'Bldg Type_TwnhsE',
       'Exter Qual_Ex', 'Exter Qual_Fa', 'Exter Qual_Gd', 'Exter Qual_TA',
       'Exter Cond_Ex', 'Exter Cond_Fa', 'Exter Cond_Gd', 'Exter Cond_Po',
       'Exter Cond_TA', 'Heat

Including cross validation on the above model

In [19]:
def transform_features(data):
    #1 drop columns with more than 5% missing values
    missing_val = data.isnull().sum()
    drop_cols = missing_val[(missing_val/len(data)) > 0.05]
    data = data.drop(columns=drop_cols.index)
    
    #2 drop text cols with more than 1 missing val
    text_cols = data.select_dtypes(include=["object"]).isnull().sum()
    text_cols_drop = text_cols[text_cols > 0]
    data = data.drop(columns=text_cols_drop.index)

    #3 fill in numerical columns with less than 5% missing values with the mode
    numerical_cols = data.select_dtypes(include=['int', 'float'])
    num_missing_val = data.isnull().sum()
    mode_replace_cols = num_missing_val[((num_missing_val/len(data)) < 0.05) &((num_missing_val/len(data)) > 0)] 
    for col in mode_replace_cols.index:
        data[col].fillna(int(data[col].mode()), inplace=True)
        
    #4 new features to built: 
    # a) how old has it been at sale YEARS_sold
    # b) how long has last remod been ago years_remod

    data["years_sold"] = data["Yr Sold"] - data["Year Built"]
    data["years_remod"] = data["Yr Sold"] - data["Year Remod/Add"]

    data = data.drop(data[data["years_sold"] < 0].index)
    data = data.drop(data[data["years_remod"] < 0].index)
    
    ## Drop columns that aren't useful for ML
    data = data.drop(["PID", "Order"], axis=1)

    ## Drop columns that leak info about the final sale
    data = data.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
    return data

def select_features(df, corr_treshold=0.4, unique_treshold=5):
    numerical_df = df.select_dtypes(include=['int', 'float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < corr_treshold].index, axis=1)
    ## Create a list of column names from documentation that are *meant* to be categorical
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

    ## Which categorical columns have we still carried with us? We'll test tehse 
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)
    
    ## How many unique values in each categorical column?
    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    ## Aribtrary cutoff of 10 unique values (worth experimenting)
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > unique_treshold].index
    df = df.drop(drop_nonuniq_cols, axis=1)

    ## Select just the remaining text columns and convert to categorical
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    
    ## Create dummy columns and add back to the dataframe!
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1)
    
    return df

def train_and_test(df, kfolds=0):
    
    
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import KFold
    
    features = df.select_dtypes(include=['int', 'float', 'uint8']).drop(columns="SalePrice").columns
    lr = LinearRegression()
    
    if kfolds == 0:
        train = df.iloc[:1460]
        test = df.iloc[1460:]

        lr.fit(train[features], train['SalePrice'])
        test_predictions = lr.predict(test[features])
        rmse = mean_squared_error(test_predictions, test['SalePrice'])**(1/2)
        return rmse
                                                                        
    if kfolds == 1:
        shuffled_index = np.random.permutation(df.index)
        df = df.reindex(shuffled_index)

        fold_one = df.iloc[0:1460].copy()
        fold_two = df.iloc[1460:].copy()
        
        lr.fit(fold_one[features], fold_one['SalePrice'])
        test_predictions = lr.predict(fold_two[features])
        rmse_1 = mean_squared_error(test_predictions, fold_two['SalePrice'])**(1/2)
        
        lr.fit(fold_two[features], fold_two['SalePrice'])
        test_predictions = lr.predict(fold_one[features])
        rmse_2 = mean_squared_error(test_predictions, fold_one['SalePrice'])**(1/2)
        rmse = (rmse_1 +rmse_2) /2
        return rmse
    
    else:
        rmse_value = []
        kf = KFold(n_splits=kfolds, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        avg_rmse = np.mean(rmse_values)
        return (avg_rmse, rmse_values)

data = pd.read_csv('../datasets/AmesHousing.tsv', delimiter="\t")
transform_df = transform_features(data)
filtered_df = select_features(transform_df, 0.4, 5)
rmse_avg, rmse_values = train_and_test(filtered_df, kfolds=10)

print(rmse_avg)

rmse_values


24911190.294953994


[25169.037789564376,
 248847914.77582255,
 24790.18142704602,
 27293.714550249057,
 29257.599087634197,
 29039.67539631784,
 23877.40225310704,
 24327.49278896487,
 48639.37223378415,
 31593.698190780753]